# Trabalho Prático 1 de Estruturas Criptográficas

  - **Autores:** (Grupo 9)
      - Nelson Faria (A84727)
      - Miguel Oliveira (A83819)

## Exercício 2

In [14]:
# Imports necessários à execução deste notebook
import os, hashlib

#from cryptography.hazmat.backends import default_backend
#from cryptography.hazmat.primitives.ciphers.aead import AESGCM

### Alínea a - Construir uma classe Python que implemente um KEM-RSA

A classe deve:
  - Inicializar cada instância recebendo  o parâmetro de segurança (tamanho em bits do módulo RSA) e gere as chaves pública e privada.
  - Conter funções para encapsulamento e revelação da chave gerada.

In [3]:
# Classe que implementa o KEM_RSA (Key Encapsulation Mechanism) baseado no RSA
# Baseamo-nos em: https://doc.sagemath.org/html/en/thematic_tutorials/numtheory_rsa.html
#                 https://docs.python.org/3/library/hashlib.html

class KEM_RSA(object):
    
    def __init__(self, N, timeout=None):
        
        # Obter um primo aleatorio para o parametro q do rsa no intervalo [2^(N-1);2^N-1]
        self.q = self.primoAleatorio(N)
        # Obter um primo aleatorio para o parametro p do rsa no intervalo [2^(N+1-1);2^(N+1)-1]
        self.p = self.primoAleatorio(N + 1)
        # n = p * q
        self.n = self.p * self.q
        # Funcao totiente de euler = (p-1)*(q-1)
        self.phi = (self.p-1)*(self.q-1)
        # Escolher um inteiro aleatorio e entre 1 < e < φ(n) e que e e φ(n) sejam relativamente primos
        self.e = ZZ.random_element(self.phi)
        while gcd(self.e, self.phi) != 1:
            self.e = ZZ.random_element(self.phi)
        
        # Para calcular d, usamos o 'extended Euclidean algorithm': de−k⋅φ(n)=1 -> Assimm, so precisamos de descobrir d e -k
        # xgcd(x, y) retorna um triplo (g, s, t) que satisfaz a identidade de Bézout: g=gcd(x,y)=sx+ty 
        self.bezout = xgcd(self.e, self.phi)
        # d = mod(s,φ(n)), uma vez que 1 < d < φ(n)
        self.d = Integer(mod(self.bezout[1], self.phi))
        
          
    # Retorna um primo aleatorio entre 2^(N-1) e 2^N - 1
    def primoAleatorio(self, N):
        
        return random_prime(2**N-1,False,2**(N-1))

    
    # Funcao que serve para encapsular a chave que for acordada a partir de uma chave publica
    def encapsula(self, e, n):
        
        # Escolher um inteiro aleatorio r entre 0 < r < n 
        r = ZZ.random_element(n)
        
        # Criptograma com este inteiro usado para o encapsulamento da chave (c ← r^e mod n)
        c = Integer(power_mod(r, e, n))
        
        # Gerar o salt para derivar a chave
        salt = os.urandom(16)
        # Geracao da chave simetrica a partir do r (W ← KDF(r))
        w = hashlib.pbkdf2_hmac('sha256', str(r).encode(), salt, 100000)
        
        return (w, salt + str(c).encode())
    
    
    # Funcao usada para desencapsular uma chave, a partir do seu "encapsulamento"
    def desencapsula(self, cs):
        
        # Buscar os 16 primeiros bytes para obter o salt e o restante é o "encapsulamento" da chave
        salt = cs[:16]
        c = int(cs[16:].decode())
        
        # Obter o r (r ← c^d mod n) com o algoritmo power_mod
        r = Integer(power_mod(c, self.d, self.n))
        
        # Geracao da chave simetrica a partir do r (W ← KDF(r))
        w = hashlib.pbkdf2_hmac('sha256', str(r).encode(), salt, 100000)
        
        return w
        

**Testagem da classe definida acima:**

In [4]:
# Parametro de seguranca
N = 1024

# Chave publica: (n,e)
# Chave privada: (p,q,d)
# Inicializacao da classe responsavel por implementar o KEM-RSA
kemrsa = KEM_RSA(N)

# Verificar que ed == 1 (mod φ(n))
#print(mod(kemrsa.e * kemrsa.d, kemrsa.phi))

# Procede-se ao encapsulamento
(w,c) = kemrsa.encapsula(kemrsa.e, kemrsa.n)

print("Chave devolvida pelo encapsulamento: ")
print(w)
print("\n'Encapsulamento' da chave: ")
print(c)

# Procede-se ao desencapsulamento
w1 = kemrsa.desencapsula(c)
print("\nChave devolvida pelo desencapsulamento: ")
print(w1)

Chave devolvida pelo encapsulamento: 
b'\xbe\x8fp\xe3\xe2I\xa3\xca\x8c\xf1\xa5\xf0\xb0R!\x1f\xaa\xe7\x1d\xf0e\xf0\xaf,\x17\x19D\xbf\x04\xf5rT'

'Encapsulamento' da chave: 
b'\x19\x10\xdb\x0c\x9e;\x02N\xec\xfd2\xdd\xae\xed\x7f\xe221022107257581224947288557096205648019279201850933679188454880369826699628577151787516501467549899306133629173196961097882905579825604186850976899614025863625491400124231164740174504940636425578697242456494002491744230151925328109926000626912358190313101086624054993376199724857116787653044832020700941989990887653238377296947988657244459896404112447253113695672447611168543089655846277841547420038055486700417825675514566082253861731660306597465307306247622189145196853316898875416448256734076756221004434612584511398359623463207113669849666232393318351535547639266799564539500712085710154048897316254638335427611821'

Chave devolvida pelo desencapsulamento: 
b'\xbe\x8fp\xe3\xe2I\xa3\xca\x8c\xf1\xa5\xf0\xb0R!\x1f\xaa\xe7\x1d\xf0e\xf0\xaf,\x17\x19D\xbf\x04\xf5rT'


### Alínea b - Construir,  a partir do KEM definido anteriormente e usando a transformação de Fujisaki-Okamoto, um PKE que seja IND-CCA seguro.

In [5]:
# Classe que implementa um PKE_IND_CCA (Public Key Encryption) a partir do KEM_RSA e da transformação de Fujisaki-Okamoto
# Baseamo-nos em: https://doc.sagemath.org/html/en/thematic_tutorials/numtheory_rsa.html
#                 https://docs.python.org/3/library/hashlib.html

class PKE_IND_CCA(object):
    
    def __init__(self, N, timeout=None):
        
        self.kem = KEM_RSA(N)
    
    
    # Funcao usada para cifrar que recebe a mensagem e uma chave publica (e,n)
    def cifra(self, m, e, n):
        
        # Obter a chave e o seu 'encapsulamento'
        (w,c) = self.kem.encapsula(e,n)
        # Cifrar a mesagem com a chave simetrica
        iv = os.urandom(12)
        encryptor = Cipher(algorithms.AES(w),
                           modes.GCM(iv),
                           backend=default_backend()).encryptor()
        ciphertext = encryptor.update(m) + encryptor.finalize()
        return (c,ciphertext,encryptor.tag,iv)
    
    
    # Funcao usada para decifrar que recebe o criptograma, o 'encapsulamento' da chave, a tag e o vetor inicializacao
    def decifra(self, ciphertext, c, tag, iv):
        
        # Fazer o desencapsulamento da chave
        w = self.kem.desencapsula(c)
        # decifrar o criptograma a partir da chave obtida
        decryptor = Cipher(algorithms.AES(w),
                           modes.GCM(iv,tag),
                           backend=default_backend()).decryptor()
        return decryptor.update(ciphertext) + decryptor.finalize()
        
    

In [2]:
pke = PKE_IND_CCA(1024)

(c,ciphertext,tag,iv) = pke.cifra("Nelson Faria", pke.kem.e, pke.kem.n)

print("Texto cifrado:")
print(ciphertext)

print("\nTexto decifrado:")
print(pke.decifra(ciphertext, c, tag, iv))

NameError: name 'PKE_IND_CCA' is not defined

### Alínea c - Construir uma classe Python que implemente o DSA (*Digital Signature Algorithm*). 

*A implementação deve:*
  - *na inicialização,  receber como parâmetros o tamanho  dos primos $p$ e $q$;* 
  - *deve conter funções para assinar digitalmente e verificar a assinatura.*

Tendo em conta os requisitos descritos acima, procedemos então em seguida à descrição de todos os passos que efetuamos para a implementação do DSA.

**Criação do par de chaves pública e privada(feito na inicialização da classe)**

Para a criação do par de chaves pública e privada $(y,x)$, é necessário primeiro criar os parâmetros $(p,q,g)$ e só depois é que se pode efetuar o cálculo das chaves pública e privada. Deste modo, a função de inicialização da classe recebe como parâmetros $L$ e $N$ que são respetivamente o tamanho em bits dos parâmetros $p$ e $q$ e depois procede à criação de todos os valores referidos anteriormente, de acordo com o que será apresentado de seguida.

- **Geração dos parâmetros $(p,q,g)$**

   - Primeiro foi necessário gerar o número primo $q$ de forma aleatória entre $[2^{N-1};2^N]$;
   - De seguida, procedeu-se ao cálculo do número primo $p$ também de forma aleatória, porém o senão é que para que o algoritmo funcione corretamente, $p-1$ têm de ser múltiplo de $q$. Existem várias formas de fazer isto, porém a forma que escolhemos será descrita de seguida:
   
   Se dissermos que $q$ é então um número primo aleatório, então p = $2*q + 1$ pode ser testado para ver se é primo, sendo que tem ainda garantida a característica de $p-1$ ser múltiplo de $q$. Dentro disto, aquilo que fizemos foi precisamente isto, mas de forma a que este valor esteja no intervalo $[2^{L-1},2^{L}]$ e, além disso seja primo e aleatório!!!
   
   Por isso, o que se pensou fazer para $p$ foi múltiplicar o 2 por um número aleatório entre $[1,2^{L-N-2}]$ e somar-lhe o limite inferior para o parâmetro $p$, pelo que depois foi só seguir a seguinte regra da matemática:
   $2^X + 2*(2^Y) <=> 2^{X+Y+1}$
   
   Se X = 47 e Y = 16, então $2^{47} + 2*(2^{16}) = 2^{47+16+1} = 2^{64}$
   
   Com isto conseguimos arranjar a seguinte solução mais ou menos eficiente: 
   
   $p := (2^{L-N-1} + 2*random(1,2^{L-N-2})) * q + 1$ 
   
   Assim, conseguimos ter um número aleatório e primo no intervalo de $[2^{L-1},2^{L}]$, visto que q têm tamanho $[2^{N-1};2^N]$;
   
   - Por fim, só falta calcular o gerador $g$, pelo que para isso foi só necessário ter em conta que este não pode ser igual a 1 ($g != 1$) e ainda a seguinte fórmula: 
   
   $g := h^{(p-1) / q}$
   
   Em que $h$ é um número inteiro aleatório entre $[1;p-1]$.

- **Geração do par de chaves pública e privada $(y,x)$**

  - Para gerar a chave privada $x$ basta gerar um número inteiro aleatório entre $[1,q-1]$;
  
  - A chave pública $y$ é o resultado da seguinte expressão:
  
   $y := g^x$ $mod$ $p$

**Assinatura digital da mensagem(função *assina()*)**

Depois de criados todos os parâmetros na inicialização da classe, bem como as chave pública e privada, já se pode assinar qualquer mensagem da escolha do utilizador. O resultado da função *assina()* é o par $(r,s)$ que representa a assinatura e é calculado da seguinte forma:

- Geração de um inteiro $k$ aleatoriamente entre $[1,q-1]$;

- Calcular o valor $r$ com base na seguinte fórmula:

 $r := (g^k$ $mod$ $p)$ $mod$ $q$, em que $r != 0$

- Calcular o valor $s$ com base na seguinte fórmula:
 
 $s := (k^{-1} (H(m) + xr))$ $mod$ $q$, em que $s != 0$

**Verificação da assinatura digital(função *verifica()*)**

Por fim, a função $verifica()$ serve para verificar se uma determinada assinatura é válida. Por isso, esta recebe como parâmetros a assinatura, a mensagem original e ainda a chave pública $y$ da entidade que assinou a mensagem. Assim, os passos que são efetuados por esta função são os seguintes:

- Primeiro a função verifica se os parâmetros $r$ e $s$ da assinatura estão entre $[0,q]$;
- Calcular $w$ que é o resultado de:
 
 $w := s^{-1} mod$ $q$
 
- Calcular $u1$ que é o resultado de:

 $u1 := H(m).w$ $mod$ $q$

- Calcular $u2$ que é o resultado de:

 $u2 := r.w$ $mod$ $q$

- Calcular $v$ que é o resultado de:

 $v := (g^{u1} . y^{u2} mod$ $p)$ $mod$ $q$

- Por fim, se $v == r$, então a assinatura é válida!

In [3]:
# Classe que implementa o algoritmo de assinatura digital(DSA) 
# Baseamo-nos em: https://en.wikipedia.org/wiki/Digital_Signature_Algorithm
#                 https://doc.sagemath.org/html/en/developer/coding_in_python.html

class DSA(object):
    
    def __init__(self, L, N, timeout=None):
        
        # Escolha de um numero primo q grande
        self.q = Integer(self.primoAleatorio(N))
        # Escolha de um numero primo p grande, em que p-1 seja multiplo de q
        self.p = 1
        while (not is_prime(self.p)):
            # Como 2^(L-1) < p < 2^L e 2^(N-1) < q < 2^N, seguimos a logica seguinte: 2^(L-N) * 2^N = 2^(L-N+N) = 2^L
            # Além disso, Se n for aleatorio, 2*n + 1 tambem é
            # p := (2^(L-N-1) + 2*random(1,2^(L-N-2))) * q + 1
            self.p = (2^(L-N-1) + 2*ZZ.random_element(1, 2^(L-N-2)))*self.q + 1  
        # calcular o gerador g tal que tal que g := h^(p-1)/q mod p, com , com g != 1
        self.g = 1
        while self.g == 1:
            # Escolher um inteiro aleatorio h entre 1 < h < p-1 
            h = ZZ.random_element(2, self.p-1)
            self.g = power_mod(h, (self.p-1) // self.q, self.p)
        # Gerar a chave privada x aleatoriamente entre 1 e q-1
        self.x = ZZ.random_element(1, self.q)
        # Gerar a chave pública y := g^x mod p
        self.y = Integer(power_mod(self.g, self.x, self.p))
    
    
    # Retorna um primo aleatorio entre 2^(N-1) e 2^N
    def primoAleatorio(self, N):
        
        return random_prime(2**N,False,2**(N-1))
    
    
    # Funcao usada para assinar uma determinada mensagem
    def assina(self, message):
        
        # Criacao do par da assinatura DSA (r,s)
        r = 0   # r != 0
        s = 0   # s != 0
        while r == 0 or s == 0:
            # Gerar um inteiro aleatorio k entre 1 e q-1
            k = ZZ.random_element(1,self.q)
            # r := (g^k mod p) mod q, com r != 0
            r = Integer(mod(power_mod(self.g, k, self.p), self.q))
            # s := (k^-1 (H(m) + xr)) mod q, com s != 0
            s = Integer(mod(inverse_mod(k, self.q) * mod((hash(message) + self.x*r), self.q), self.q))
    
        # Assinatura = (r,s)
        return (r,s)
    
    
    # Funcao que verifica uma determinada assinatura a partir da mensagem original e da chave publica 
    def verifica(self, signature, message, y):
        
        # Retirar os valores de r e s
        r = signature[0]
        s = signature[1]
        # Verificar que os valores de r e s estao entre 0 e q
        if r > 0 and r < self.q and s > 0 and s < self.q:
            
            # Calcular w := s^-1 mod q
            w = Integer(inverse_mod(s, self.q))
            # Calcular u1 := H(m).w mod q
            u1 = Integer(mod(hash(message) * w, self.q))
            # Calcular u2 := r.w mod q
            u2 = Integer(mod(r * w, self.q))
            # Calcular v := (g^u1 * y^u2 mod p) mod q
            v = Integer(mod(mod(power_mod(self.g, u1, self.p) * power_mod(y, u2, self.p), self.p), self.q))
            print("V = " + str(v))
        
        else:
            return False
        
        return v == r
        

**Testagem da classe definida acima:**

In [4]:
# Tamanhos para os parametros p e q
L = 2048
N = 256

dsa = DSA(L,N)

print("P = " + str(dsa.p))
print("Q = " + str(dsa.q))

# Teste de uma assinatura e verificacao
message = input("\nInsira a mensagem a assinar:")
assinatura = dsa.assina(message.encode("utf-8"))
print("\nR = " + str(assinatura[0]))
print("S = " + str(assinatura[1]))
if dsa.verifica(assinatura, message.encode("utf-8"), dsa.y):
    print("A assinatura é válida!!!!")
else:
    print("A assinatura é inválida!!!!")


P = 16687928646511162233330390597464318490993651037632378453374622666085593999429932571214784162079984723656006640507022853470599499102082872898837519400881076034865354064016444272529288560010688711581667085952781324870365228454749642339827233214549077133017412519746344987422812841499144190880534693492096449568748027713654194190452359004190252499373199827695963645261664378579642964810445964221239297073650356480365378518719612636389825046836665952929571201691779433484172088955208416019081969062586451900654478238160725087323890355920660003715336396670557886490148056330145436737620315758500058368644205354178735157531
Q = 69704559910649020601246875415187208808091781318716515231358807997332044177137

Insira a mensagem a assinar:Estruturas Criptograficas

R = 36591313615978199591476434758077544143628516449149741999757507506880608120403
S = 28075705307318381511498171594985542383819068888462939884756252939102772948423
V = 365913136159781995914764347580775441436285164491497419997575075068806081

### Alínea d -  Construir uma classe Python que implemente o ECDSA (*Elliptic Curve Digital Signature Algorithm*) usando uma das curvas elípticas primas definidas no FIPS186-4  (escolhida  na iniciação da classe).

Documento em que nos baseamos: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.186-4.pdf

Mais referencias: https://pt.wikipedia.org/wiki/ECDSA

De acordo com o ficheiro mencionado acima da *FEDERAL INFORMATION PROCESSING STANDARDS PUBLICATION* (**FIPS PUB 186-4**) sobre standards de assinaturas digitais, existem várias opções para a escolha da curva elítica prima. A curva elítica que escolhemos foi a **P-256**, sendo que esta possuí os seus próprios valores standard para os parâmetros da curva descritos nesse mesmo documento. Estes parâmetros assumem um papel fundamental e representam, de forma sucinta, a curva, o ponto base da curva $G$ e ainda $n$, que é a ordem inteira de $G$ (notar que $n*G = O$, sendo $O$ o elemento identidade). Assim, procedemos aos vários passos do DSA, mas em modo de curvas elíticas.

**Criação do par de chaves pública e privada(feito na inicialização da classe)** (baseado na pág 62)

A chave pública $Q$, a chave privada $d$  e os parâmetros do domínio da curva elítica estão matemáticamente relacionados uns com os outros. Deste modo, para a criação deste par de chaves é necessário:

  - Chave Privada $d$: é um inteiro gerado de forma aleatória no intervalo de [1,n-1]
  - Chave Pública $Q$: dado um ponto gerador (ou ponto base) $G$ da curva elítica, $Q$ é um ponto na curva elítica que obedece à seguinte relação: $Q := d * G$
  
**Assinatura digital da mensagem(função *assina()*)** (baseado na página da wikipedia)

De seguida serão descritos todos os passos que foram efetuados para assinar $(r,s)$ um dada mensagem recebida como input.

   - Calcular $e = HASH(m)$ (no nosso caso a função de hash que vamos usar é o SHA-256)
   - Considerar $z$ como sendo os $L_{n}$ bits mais à esquerda de $e$, em que $L_{n}$ é o comprimento de bit da ordem do grupo $n$. (Nota só para o facto de que $z$ pode ser maior do que $n$ mas não mais longo. Além disso, referir o facto de como o hash que usamos é o SHA-256 e a curva possuí um $n$ de 256 bits, então no nosso caso $z==e$)
   - Geração de um número inteiro aleatório $k$ entre o intervalo [1,n-1];
   - Calcular um ponto na curva tal que: $(x1, y1) = k * G$, em que $G$ continua a ser um ponto gerador da curva que foi definido inicialmente;
   - Calcular então o valor de $r$ tal que $r = x1$ $mod$ $n$. Se $r = 0$, então voltar a gerar um $k$ e consequentemente o ponto $(x1,y1)$;
   - Por fim, calcular o valor de $s$ que é igual a $s = k^{-1} * (z + r*d)$ $mod$ $n$. Se $s = 0$, então voltar a gerar um $k$ e consequentemente o ponto $(x1,y1)$;
   - A assinatura é o par $(r,s)$. $(E (r,-s$ $mod$ $n)$ também é uma assinatura válida.)

Um dos pontos importantes é a geração do número aleatório $k$. Se duas mensagens diferentes utilizarem na geração um $k$ igual, podem ocorrer ataques que podem resultar na extração da chave de assinatura. Assim, para garantir que $k$ é único para cada mensagem, pode-se ignorar completamente a geração de números aleatórios e gerar assinaturas determinísticas derivando $k$ tanto da mensagem quanto da chave privada.

**Verificação da assinatura digital(função *verifica()*)** (baseado na página da wikipedia)

Antes de mais nada, para efetuar a verificação da assinatura de uma dada mensagem, deve-se ter uma cópia da chave pública de ponto da curva $Q$. Nós podemos verificar que $Q$ é um ponto válido da curva da seguinte forma:

   - Verificar que $Q$ não é igual ao elemento identidade $O$, e as suas coordenadas são, de outra forma válida;
   - Verificar que $Q$ está sobre a curva;
   - Verificar que $n * Q = O$.

Depois de verificarmos isto, podemos seguir para os passos seguintes de verificação da assinatura:

   - Verificar que $r$ e $s$ são números inteiros em $[1,n-1]$. Se não, a assinatura é inválida;
   - Calcular $e = HASH(m)$ (este hash que é feito aqui deve ser o mesmo que foi usado na assinatura da mensagem);
   - Considerar $z$ como os $L_{n}$ bits mais à esquerda de $e$;
   - Calcular $w=s^{-1} $$mod$ $n$;
   - Calcular $u_{1}=zw$ $mod$ $n$;
   - Calcular $u_{2}=rw$ $mod$ $n$;
   - Calcular o ponto da curva $(x_{1},y_{1})=u_{1} * G+u_{2} * Q$. Se $(x_{1},y_{1})=O$, então a assinatura é inválida.
   - A assinatura é válida se $r\equiv x_{1}$ $mod$ $n$, e inválida caso contrário.

In [12]:
# Classe que implementa o Algoritmo de Assinatura Digital de Curvas Elípticas(ECDSA)
# Baseado no documento(durante o codigo, tem referencias a paginas deste documento): 
# https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.186-4.pdf
# https://doc.sagemath.org/html/en/reference/finite_rings/sage/rings/finite_rings/finite_field_constructor.html
# https://doc.sagemath.org/html/en/reference/arithmetic_curves/sage/schemes/elliptic_curves/constructor.html

class ECDSA(object):
    
    def __init__(self, timeout=None):
        
        # Parâmetros da Curva Elíptica P-256 (ver pag. 91)
        # p e n são números primos de 256 bits
        self.p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
        self.n = 115792089210356248762697446949407573529996955224135760342422259061068512044369
        # A seleção a ≡ -3 para o coeficiente de x foi feita por razões de eficiência(ver pag.89); consulte IEEE Std 1363-2000
        self.a = -3
        # Este parametro esta em hexadecimal
        self.b = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b
        # Este parametro esta em hexadecimal
        self.Gx = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
        # Este parametro esta em hexadecimal
        self.Gy = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5
        
        # Criacao do corpo finito em torno do numero primo p
        Fp = GF(self.p)
        # Equacao de Weierstrass:  y^2 + a_1xy + a3y = x^3 + a2^2 + a4x + a6
        # Curva Elíptica E sobre o corpo finito Fp, onde a1=a2=a3=0, a4 = a mod p e a6 = b mod p
        self.E = EllipticCurve(Fp, [self.a, int(self.b)])
        # Ponto Gerador G pertencente à Curva Elíptica E
        self.G = self.E((self.Gx,self.Gy))
        
        # Criacao do par de chaves publica e privada (Q,d)
        # Chave privada -> inteiro gerado de forma aleatória no intervalo de [1,n-1]
        self.d = ZZ.random_element(1, self.n)
        # Chave publica -> Q := d * G
        self.Q = self.d * self.G
        
    
    # Funcao usada para assinar uma determinada mensagem
    def assina(self, message):
        
        # Calcular e=HASH(m) 
        e = Integer('0x' + hashlib.sha256(message).hexdigest())
        # Criacao do corpo finito em torno do numero primo n
        Fn = GF(self.n)
        # r e s nao podem ser 0, sendo unicamento inicializado a 0 para controlar o ciclo
        r = 0
        s = 0
        while r==0 or s==0:
            
            # Geracao de um numero aleatorio k no intervalo de [1,n-1]
            k = ZZ.random_element(1, self.n)
            # (x1,y1)=k∗G
            (x1,y1) = (k * self.G).xy()
            # Calcular r tal que r=x1  mod n, em que r!=0 (Decimos aplicar um corpo finito em torno de n, que vai ter ao mesmo)
            r = Integer(Fn(x1))
            # Calcular s tal que s=k−1∗(z+r∗d) mod n, em que s!=0. No nosso caso, z==e
            s = Fn(k^-1 * (e + r*self.d))
        
        # Assinatura = (r,s)
        return (r,s)
    
    
    # Funcao que verifica uma determinada assinatura a partir da mensagem original e da chave publica
    def verifica(self, signature, message, Q):
        
        # Antes de verificar a assinatura, verificar o Q recebido como parâmetro (sabendo que n*G = O)
        if Q == self.n*self.G or self.n*Q != self.n*self.G:
            return False
        # Verificar de Q pertence à curva
        try:
            self.E(Q)
        except TypeError as e:
            return False
        # Retirar os valores de r e s
        r = signature[0]
        s = signature[1]
        # Verificar que r e s são números inteiros em [1,n−1]. Se não, a assinatura é inválida
        if r>=1 and r<=self.n-1 and s>=1 and s<=self.n-1:
            
            # Calcular e=HASH(m), que deve ser feito da mesma forma que foi feito na assinatura
            e = Integer('0x' + hashlib.sha256(message).hexdigest())
            # Criacao do corpo finito em torno do numero primo n
            Fn = GF(self.n)
            # Calcular w=s−1 mod n
            w = Fn(s^-1)
            # Calcular u1=zw mod n, no nosso caso, z==e
            u1 = Integer(Fn(e*w))
            # Calcular u2=rw mod n
            u2 = Integer(Fn(r*w))
            # Calcular (x1,y1)=u1∗G+u2∗Q
            XY = u1*self.G + u2*self.Q
            (x1,y2) = XY.xy()
            if XY == self.n*self.G:
                return False
            
        else:
            return False
        
        print("Fn(x1): " + str(Fn(x1)))
        
        return r == Fn(x1)
        

**Testagem da classe definida acima:**

In [15]:
# Iniciar a classe
ecdsa = ECDSA()

print("Chave Pública Q = " + str(ecdsa.Q))
print("Chave Privada d = " + str(ecdsa.d))

# Teste de uma assinatura e verificacao
message = input("\nInsira a mensagem a assinar:")
assinatura = ecdsa.assina(message.encode("utf-8"))
print("\nR = " + str(assinatura[0]))
print("S = " + str(assinatura[1]))
if ecdsa.verifica(assinatura, message.encode("utf-8"), ecdsa.Q):
    print("A assinatura é válida!!!!")
else:
    print("A assinatura é inválida!!!!")

Chave Pública Q = (57878228548902560094143304512948357408965790511123964189419882050200519572919 : 83400247987107374355766010925905011640597158924692079622058424675246602103809 : 1)
Chave Privada d = 16310054232640418519365995138743185605773818078825052747007614626484348284017

Insira a mensagem a assinar:Estruturas Criptograficas

R = 3493842437035606191148725861828962271375208793379474918741241426794018778204
S = 21957216170437439821858369310225161521729922191149245993343162027337351548508
Fn(x1): 3493842437035606191148725861828962271375208793379474918741241426794018778204
A assinatura é válida!!!!


**O que esta a baixo foi so usado para testar enquanto se fazia o codigo**

In [141]:
p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
n = 115792089210356248762697446949407573529996955224135760342422259061068512044369
# A seleção a ≡ -3 para o coeficiente de x foi feita por razões de eficiência(ver pag.89); consulte IEEE Std 1363-2000
a = -3
# Este parametro esta em hexadecimal
b = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b
# Este parametro esta em hexadecimal
Gx = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
Gy = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5

# Criacao do corpo finito em torno do numero primo p
Fp = GF(p)
# Equacao de Weierstrass:  y^2 + a_1xy + a3y = x^3 + a2^2 + a4x + a6
# Curva Elíptica E sobre o corpo finito Fp, onde a1=a2=a3=0, a4 = a mod p e a6 = b mod p
E = EllipticCurve(Fp, [a, int(b)])
# Ponto Gerador G pertencente à Curva Elíptica E
G = E(Gx,Gy)

print(G)

d = ZZ.random_element(1, n)
# Chave publica -> Q := d * G
Q = d * G

print(d)
print(Q)

Fn = GF(n)
k = ZZ.random_element(1, n)
e = Integer('0x' + hashlib.sha256(b"Nelson").hexdigest())
(x1,y1,l) = k * G
r = Fn(x1)
s = Fn(k)^(-1) * (e + (r*d))
w = Fn(s^-1)
# Calcular u1=zw mod n, no nosso caso, z==e
u1 = Integer(Fn(e*w))
# Calcular u2=rw mod n
u2 = Integer(Fn(r*w))
I = u1*G
U = (u2*Q)
P = u1*G + (u2*Q)

print(P)
print(P.xy())
print(n * Q)
print(n * G)
print(Q != n*G)

try:
    E(Q-G/7)
except TypeError as e:
    print("Ola")

(48439561293906451759052585252797914202762949526041747995844080717082404635286 : 36134250956749795798585127919587881956611106672985015071877198253568414405109 : 1)
64764433736378911609383027468775850004321411368511634099974414406913602036404
(96329905282923523670773425767487256645822277174055698677334922194922888387799 : 35184277405363381935918298199339261970409857779165953306001129918777389768479 : 1)
(73015315955985244871978173278400868816438462238849626218346606711791713151601 : 67412148190889784829947528640363189717997552952180532878252794968746303454157 : 1)
(73015315955985244871978173278400868816438462238849626218346606711791713151601, 67412148190889784829947528640363189717997552952180532878252794968746303454157)
(0 : 1 : 0)
(0 : 1 : 0)
True
Ola


In [47]:
import hashlib

message = b"Nelson Faria"
e = Integer('0x' + hashlib.sha256(message).hexdigest())

print(e)

13543130568159233805324317362418584210927670289778620905227662235190565188472


In [55]:
Fn = GF(115792089210356248762697446949407573529996955224135760342422259061068512044369)

print(Fn.order().nbits())

print(Fn(13) == mod(13,7))

256
False
